# EP1 - Modelo treinado

Felipe
Nilton

## Descricao

In [20]:
#!pip install -r requirements.txt
import pandas as pd
import numpy as np

In [21]:
np.random.seed(777)

In [22]:
df = pd.read_excel('ep1_esic2023_clareza_TRAIN.xlsx')
df.head()

,resp_text,clarity
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   resp_text  6000 non-null   object
 1   clarity    6000 non-null   object
dtypes: object(2)
memory usage: 93.9+ KB


In [24]:
df.describe()

,resp_text,clarity
count,6000,6000
unique,5626,3
top,"Prezado(a) Senhor(a), Sua manifestação foi a...",c5
freq,41,2000


In [25]:
df.groupby('clarity').describe()

resp_text                                                            \
            count unique                                                top   
clarity                                                                       
c1           2000   1839   Prezado(a) Senhor(a),  Sua manifestação foi a...   
c234         2000   1929   Prezado(a) Senhor(a),  Sua manifestação foi a...   
c5           2000   1911   Prezado (a) Senhor (a)     1. Em atenção ao p...   

              
        freq  
clarity       
c1        24  
c234      17  
c5         8

## Pre-processing

### I. Cleaning

In [26]:
import string

In [27]:
texts = df['resp_text']
texts = texts.str.lower()
texts = texts.str.translate(str.maketrans('', '', string.punctuation))
#texts = texts.str.replace('[\d+]', '') #remove numeros

df['tokens'] = texts

### II. Tokenize

In [28]:
from nltk.tokenize import word_tokenize

In [29]:
df['tokens'] = [word_tokenize(resp) for resp in df['tokens']]
df.head(10)

,resp_text,clarity,tokens
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,"[prezado, sr, jose, taunai, em, atenção, ao, s..."
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,"[a, pedido, do, próreitor, de, graduação, info..."
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e..."
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e..."
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,"[prezado, prof, gilberto, tadeu, reis, da, sil..."
5,"""Prezado Sr. Luis Cesar Lopes Zeredo, Em at...",c1,"[prezado, sr, luis, cesar, lopes, zeredo, em, ..."
6,"""Prezado Sr. Wellington de Almeida Em atendi...",c234,"[prezado, sr, wellington, de, almeida, em, ate..."
7,"""Prezado(a) Sr.(a), Agradecemos o contato e ...",c1,"[prezadoa, sra, agradecemos, o, contato, e, in..."
8,"""Prezado(a), Em atenção ao seu pedido, en...",c5,"[prezadoa, em, atenção, ao, seu, pedido, encam..."
9,"""Prezado(a), Em atenção ao seu pedido, en...",c5,"[prezadoa, em, atenção, ao, seu, pedido, encam..."


### III. Lemma

In [30]:
import spacy
import os

In [31]:
try:
    nlp = spacy.load("pt_core_news_sm")
except Exception:
    os.system("python -m spacy download pt_core_news_sm")
    nlp = spacy.load("pt_core_news_sm")

In [39]:
filtered = []
for sent in df['tokens']:
    sent = str(sent)
    doc = nlp(sent)
    temp = [token.lemma_ for token in doc if not token.is_punct]
    temp = " ".join(temp)
    filtered.append(temp)

In [47]:
df['lemma'] = filtered
df.head()

,resp_text,clarity,tokens,lemma,ngrams
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,"[prezado, sr, jose, taunai, em, atenção, ao, s...",prezar sr jose taunai em atenção a o seu pedir...,prezar sr jose taunai em atenção a o seu pedir...
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,"[a, pedido, do, próreitor, de, graduação, info...",o pedir de o próreitor de graduação informamo ...,o pedir de o próreitor de graduação informamo ...
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e...",prezar o sr o agradecer o contato e informamo ...,prezar o sr o agradecer o contato e informamo ...
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e...",prezar o sr o agradecer o contato e informamo ...,prezar o sr o agradecer o contato e informamo ...
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,"[prezado, prof, gilberto, tadeu, reis, da, sil...",prezar prof gilberto tader rei de o Silva em a...,prezar prof gilberto tader rei de o Silva em a...


In [48]:
df.to_csv('./DfCleaned.csv')

### IV N-gram

In [44]:
import nltk
from nltk.util import ngrams

In [62]:
temp = []
for txt in df['ngrams']:    
    txt = ngrams(sequence=nltk.word_tokenize(txt), n=3)
    temp.append(txt)
    

In [64]:
df['ngrams'] = temp
df.head()

,resp_text,clarity,tokens,lemma,ngrams
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,"[prezado, sr, jose, taunai, em, atenção, ao, s...",prezar sr jose taunai em atenção a o seu pedir...,<zip object at 0x7fe7d44c5dc0>
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,"[a, pedido, do, próreitor, de, graduação, info...",o pedir de o próreitor de graduação informamo ...,<zip object at 0x7fe7d44c6ac0>
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e...",prezar o sr o agradecer o contato e informamo ...,<zip object at 0x7fe7d4454580>
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,"[prezado, a, sr, a, agradecemos, o, contato, e...",prezar o sr o agradecer o contato e informamo ...,<zip object at 0x7fe7d4631a00>
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,"[prezado, prof, gilberto, tadeu, reis, da, sil...",prezar prof gilberto tader rei de o Silva em a...,<zip object at 0x7fe7d44a7b40>


In [65]:
df.to_csv('./DfNgrams.csv')